In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

from utils_camera import MyRealSense, CVCamera
import cv2
from matplotlib import pyplot as plt
import pickle
import datetime
import os 
import argparse

import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings

import matplotlib.pyplot as plt

import os
import time 

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/carl/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
dir="/home/carl/data_sawyer/models/"
fn="model_epoch_600.pth"
ckpt_path=dir+fn
assert os.path.exists(ckpt_path)

In [3]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)

# restore policy
policy, ckpt_dict = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path, device=device, verbose=True)
pass 

============= Loaded Config =============
{
    "algo_name": "bc",
    "experiment": {
        "name": "sawyer_block",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": false,
            "wandb_proj_name": "block"
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 20,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 500,
        "validation_epoch_every_n_steps": 50,
        "env": null,
        "additional_envs": null,
        "render": false,
        "render_video": true,
        "keep_all_videos": false,
        "video_skip": 5,
        "rollout": {
            "enabled": false,
            "n": 50,
            "horizon": 1000,
            "rate": 20,
            "w

/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


============= Loaded Policy =============
ObservationKeyToModalityDict: mean not found, adding mean to mapping with assumed low_dim modality!
ObservationKeyToModalityDict: scale not found, adding scale to mapping with assumed low_dim modality!
ObservationKeyToModalityDict: logits not found, adding logits to mapping with assumed low_dim modality!
BC_RNN_GMM (
  ModuleDict(
    (policy): RNNGMMActorNetwork(
        action_dim=7, std_activation=softplus, low_noise_eval=True, num_nodes=5, min_std=0.0001
  
        encoder=ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=agentview_image
                    shape=[3, 84, 84]
                    modality=rgb
                    randomizer=CropRandomizer(input_shape=[3, 84, 84], crop_size=[76, 76], num_crops=1)
                    net=VisualCore(
                      input_shape=[3, 76, 76]
                      output_shape=[64]
                      backbone_net=Res

In [4]:

ckpt_dict = FileUtils.maybe_dict_from_checkpoint(ckpt_path=ckpt_path, ckpt_dict=None)
# algo name and config from model dict
algo_name, _ = FileUtils.algo_name_from_checkpoint(ckpt_dict=ckpt_dict)
config, _ = FileUtils.config_from_checkpoint(algo_name=algo_name, ckpt_dict=ckpt_dict, verbose=False)

config.unlock()
config.train.data="/home/carl/data_sawyer/block/demos.hdf5"
config.lock()

ObsUtils.initialize_obs_utils_with_config(config)
dataset_path = os.path.expanduser(config.train.data)
shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=True
)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: ['robot0_eye_in_hand_image', 'agentview_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
obs key agentview_image with shape (84, 84, 3)
obs key robot0_eef_pos with shape (6,)
obs key robot0_eye_in_hand_image with shape (84, 84, 3)


In [5]:
def demo2actions(hdf5_file, demo_key, rollout_policy): 
    rollout_policy.start_episode()


    demo=hdf5_file['data'][demo_key]
    actions_real=demo['actions']
    obs=demo['obs']
    N=obs['robot0_eye_in_hand_image'].shape[0]

    obss=[]
    for i in range(N):
        ob={k:obs[k][i] for k in obs.keys()} 
        ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
        ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)
        obss.append(ob)

    acs=[]
    for ob in obss: 
        ob = rollout_policy._prepare_observation(ob)
        ac = rollout_policy.policy.get_action(obs_dict=ob, goal_dict=None)
        ac=ac.detach().cpu().numpy()[0]
        acs.append(ac)

    actions_pred=np.array(acs)
    return actions_real, actions_pred

In [6]:

class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None
        self.latest_gripper_state=False
        self.record_msg=None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)
    
    def gripper_callback(self, msg):
        self.latest_gripper_state=msg.data
    def android_callback(self, msg):
        self.record_msg = msg

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        self.node.create_subscription(Bool, '/gripper_command', self.gripper_callback, 10)
        self.node.create_subscription(Bool, '/android_record', self.android_callback, 10)
        rclpy.spin(self.node)


In [8]:
rclpy.init()

urdf_path = os.getcwd() +'/'+'sawyer.urdf'  #'/home/carl/sawyer_robot_ros2/src/teleop_script/sawyer.urdf'
robot = URDFModel(urdf_path)
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}


joint_names=['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
joint_inds=[jointMap[name] for name in joint_names]


node = Node('infer_live')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [9]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

def set_vels(vels):
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =vels 
    cmd_pub.publish(cmd_msg)

def goto_xyzrpy(target, delay=0.1, tol=0.05):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target
    xd[3:6]=0 

    # print('xd: ', xd)

    alpha = .0001
    I = np.eye(q.shape[0])

    while np.linalg.norm(xd)>tol:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 
        xd[3:6]=0        # only xyz, rpy not working yet

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        set_vels(list(qd))
        time.sleep(delay)
    
    # stop
    set_vels([0.0]*10)

apos=0
def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1):
    global apos
    target_jointpos=target_q
    cpos=robot.getJoints()
    dpos=np.linalg.norm(target_jointpos-cpos)

    while dpos>tol: 
        cpos=robot.getJoints()
        dpos=np.linalg.norm(target_jointpos-cpos)
          

        qd =(target_jointpos-cpos)*alpha
        apos=qd 
        if np.max(qd) <0.001:
            print('reached')
            break 


        set_vels(list(qd))
        time.sleep(delay)
        
    set_vels([0.0]*10)

### Robot safety check

In [10]:
pos1=robot.getJoints()
pos1

array([ 0.25362402, -0.29735156,  0.2595332 , -1.92522363,  1.4523877 ,
       -1.24452832, -1.88056152,  0.57152637,  0.041667  ,  0.        ])

In [11]:
goto_q_pos_velocity_ctrl(pos1, tol=0.11)

### set robot to initial pos

In [12]:
block_start_pos = np.array([ 0.25362402, -0.1616416 ,  0.2595332 , -1.92522363,  1.4523877 ,
       -1.24452832, -1.8799541 ,  0.57152539,  0.03735149,  0.        ])

In [15]:
goto_q_pos_velocity_ctrl(block_start_pos, tol=0.15)

### Robot is ready

In [52]:
policy.start_episode()

In [22]:
!v4l2-ctl --list-devices

Streaming Camera W2G: Streaming (usb-0000:00:14.0-3.1):
	/dev/video2
	/dev/video4
	/dev/media1

USB2.0 HD UVC WebCam: USB2.0 HD (usb-0000:00:14.0-9):
	/dev/video0
	/dev/video1
	/dev/media0

Intel(R) RealSense(TM) Depth Ca (usb-0000:3a:00.0-1):
	/dev/video5
	/dev/video6
	/dev/video10
	/dev/video15
	/dev/video16
	/dev/video17
	/dev/media2
	/dev/media3



In [23]:
camera=2

cam_wrist=MyRealSense()
cam_front=CVCamera(camera)

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (1100) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [26]:
def get_current_obs():
    img_wrist=cam_wrist.get_current_frame(scale=0.5)  
    img_front=cam_front.get_current_frame(scale=0.5) 
    # print('shape: ', image_wrist.shape, image_front.shape)
    msg=ros_interface.latest_joint_states
    gripper_isopen=ros_interface.latest_gripper_state

    ee=get_ee(msg)


    pos =np.array([msg.position[ind] for ind in joint_inds])
    vel =np.array([msg.velocity[ind] for ind in joint_inds])
    t=msg.header.stamp.sec+msg.header.stamp.nanosec*1e-9

    img_wrist=cv2.cvtColor(img_wrist, cv2.COLOR_BGR2RGB)
    img_front=cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)

    img_wrist=cv2.resize(img_wrist, (84,84))
    img_front=cv2.resize(img_front, (84,84))


    ob={'agentview_image': img_front, 'robot0_eye_in_hand_image': img_wrist, 'robot0_joint_pos': pos, 'robot0_joint_vel':vel, 'robot0_eef_pos': ee}
    ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
    ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)

    return ob

In [27]:
ob=get_current_obs()
ob = policy._prepare_observation(ob)
ac = policy.policy.get_action(obs_dict=ob, goal_dict=None)
ac=ac.detach().cpu().numpy()[0]
ac 

array([ 5.6670530e-04, -3.4464477e-04,  3.2395394e-05, -9.1558039e-02,
       -4.9376651e-04, -9.1197424e-02,  9.9992144e-01], dtype=float32)

In [29]:
ob=get_current_obs()
ob = policy._prepare_observation(ob)
ac = policy.policy.get_action(obs_dict=ob, goal_dict=None)
ac=ac.detach().cpu().numpy()[0]
ac 

array([ 1.1529961e-03, -8.4776297e-04,  2.0385985e-04, -9.5431827e-02,
       -1.2647170e-03, -9.5166475e-02,  1.0000403e+00], dtype=float32)

### Robot Finger needs to be fixed